# **作業說明**
# (這是Udacity關於A/B Test的期末專題)

Udacity希望了解，在免費14天試學網頁上，除了要信用卡資訊外，還想了解學生願意花多少小時學。如果少於某門檻(5小時)，就建議學生不要註冊，免費聽聽影音就好，免得浪費資源，降低學習成功率。

我們的題目是，增加這個頁面，是否對Gross Conversion(GC)和Net Conversion (NC)在統計學上(Alpha=0.05，Power=0.8)有幫助(d=0.01/0.0075)，亦即統計上的顯著(Significant)。

CI = click 數目

GC = 註冊數/CI (聽了建議仍然註冊的比例)

NC = 繳費數/CI (14天之後繳費且繼續的比例)

我們期待GC比原來下降，但NC不降，這表示省去資源但收入不降。

檔名：ab-tests-with-python.ipynb

**作業目標**

1.   經由範例程式，學習A/B Test 的步驟
2.   最低樣本數的計算方法
3.   自行開發信賴區間計算函數













In [3]:
#載入程式庫
import math as mt
import numpy as np
import pandas as pd
from scipy.stats import norm

In [32]:
#推算值：                 /      NConversion      \
#邏輯架構：進入    →    點擊    →    註冊    →    付費
#推算值：     \  CTP   / \GConversion/ \ Retention /

# 將基礎數據放入字典
# Cookies(進入數，人次)、Clicks(點擊數)、Enrollments(註冊數)
# CTP(點擊數/Cookies)、G轉化率(註冊數/點擊數)
# N轉化率(付費/點擊數)、滯留數(付費/註冊數)
baseline = {"Cookies":40000,"Clicks":3200,"Enrollments":660,
            "CTP":0.08,"GConversion":0.20625,
           "NConversion":0.109313,"Retention":0.53}

In [33]:
#調整大小到以Cookie為基準
baseline["Cookies"] = 5000
baseline["Clicks"]=baseline["Clicks"]*(5000/40000)
baseline["Enrollments"]=baseline["Enrollments"]*(5000/40000)
baseline

{'Cookies': 5000,
 'Clicks': 400.0,
 'Enrollments': 82.5,
 'CTP': 0.08,
 'GConversion': 0.20625,
 'NConversion': 0.109313,
 'Retention': 0.53}

In [34]:
# 算出 Gross Conversion (GC) 的 p、n和Standard Deviation(sd) rounded to 4 decimal digits.，d_min為業績上的最小變化
# 沒有使用pd.Series 或 pd.DataFrame，則也可以用dict不指定key來含括
GC={}
GC["d_min"]=0.01
GC["p"]=baseline["GConversion"]
GC["n"]=baseline["Clicks"]
#二項分配（binom）的抽樣，注意會除n
#p = 某值; n = 重複次數; q = 1-p; 變異數 = pq/n
#此處標準差為standard error of the proportion 所以又稱se
GC["sd"]=round(mt.sqrt((GC["p"]*(1-GC["p"]))/GC["n"]),4)
#當然開根號也可以**0.5，
#或是**(2**-1)，表示方式會對數較接近
GC["test"]=round((GC["p"]*(1-GC["p"])/GC["n"])**0.5,4)
GC

{'d_min': 0.01, 'p': 0.20625, 'n': 400.0, 'sd': 0.0202, 'test': 0.0202}

In [35]:
# Retention(R) 

R={}
R["d_min"]=0.01
R["p"]=baseline["Retention"]
R["n"]=baseline["Enrollments"]
R["sd"]=round(((R["p"]*(1-R["p"]))/R["n"])**0.5,4)
R

{'d_min': 0.01, 'p': 0.53, 'n': 82.5, 'sd': 0.0549}

In [36]:
# Net Conversion (NC)
NC={}
NC["d_min"]=0.0075
NC["p"]=baseline["NConversion"]
NC["n"]=baseline["Clicks"]
NC["sd"]=round(((NC["p"]*(1-NC["p"]))/NC["n"])**0.5,4)
NC

{'d_min': 0.0075, 'p': 0.109313, 'n': 400.0, 'sd': 0.0156}

In [38]:
#計算 Z-score，以百分點函數獲得該機率(probability)對應的z值(norm為常態分佈)
def get_z_score(alpha):
    return norm.ppf(alpha)

# 得到兩個(A/B)標準差
def get_sds(p,d):
    sd1=mt.sqrt(2*p*(1-p))
    sd2=mt.sqrt(p*(1-p)+(p+d)*(1-(p+d)))
    sds=[sd1,sd2]
    return sds

# 求Sample Size，不過有個奇怪的假設是，虛無假設H0：Pcont = Pexp 沒問題，對立假設以對照會比試驗組(處理組)較好進行假設
# Pcont - Pexp = d，其中d是detectable effect。一般會認為新制要抗衡舊制必須要提出較佳解方可取代，不過也不會影響結果(僅影響正負值)
# 往下，根據公式(依據假設的前提，為單尾，alpha要除二)
def get_sampSize(sds,alpha,beta,d):
    n=pow((get_z_score(1-alpha/2)*sds[0]+get_z_score(1-beta)*sds[1]),2)/pow(d,2)
    return n

In [39]:
GC["d"]=0.01
R["d"]=0.01
NC["d"]=0.0075

In [40]:
# Let's get an integer value for simplicity, round()
GC["SampSize"]=round(get_sampSize(get_sds(GC["p"],GC["d"]),0.05,0.2,GC["d"]))
GC["SampSize"]

25835

In [41]:
#CTP = 0.08
GC["SampSize"]=round(GC["SampSize"]/baseline["CTP"]*2)
GC["SampSize"]

645875

In [42]:
# Getting a integer value
R["SampSize"]=round(get_sampSize(get_sds(R["p"],R["d"]),0.05,0.2,R["d"]))
R["SampSize"]

39087

In [43]:
# GConversion = 0.20625
R["SampSize"]=R["SampSize"]/baseline["CTP"]/baseline["GConversion"]*2
R["SampSize"]

4737818.181818182

In [44]:
# Getting a integer value
NC["SampSize"]=round(get_sampSize(get_sds(NC["p"],NC["d"]),0.05,0.2,NC["d"]))
NC["SampSize"]

27413

In [45]:
NC["SampSize"]=NC["SampSize"]/baseline["CTP"]*2
NC["SampSize"]

685325.0

In [199]:
# 載入數據
control=pd.read_csv("control_data.csv")
experiment=pd.read_csv("experiment_data.csv")
# display(control.head())
# display(experiment.head())

In [47]:
pageviews_cont=control['Pageviews'].sum()
pageviews_exp=experiment['Pageviews'].sum()
pageviews_total=pageviews_cont+pageviews_exp
print ("number of pageviews in control:", pageviews_cont)
print ("number of Pageviewsin experiment:" ,pageviews_exp)

number of pageviews in control: 345543
number of Pageviewsin experiment: 344660


In [8]:
# Count the total clicks from complete records only
# 想根據Enrollments排點擊數，但是Enrollments下面有NAN，需要notsull()
clicks_cont=control["Clicks"].loc[control["Enrollments"].notnull()].sum()
clicks_exp=experiment["Clicks"].loc[experiment["Enrollments"].notnull()].sum()
clicks_cont

17293

In [49]:
#Gross Conversion - number of enrollments divided by number of clicks
enrollments_cont=control["Enrollments"].sum()
enrollments_exp=experiment["Enrollments"].sum()

GC_cont=enrollments_cont/clicks_cont
GC_exp=enrollments_exp/clicks_exp
GC_pooled=(enrollments_cont+enrollments_exp)/(clicks_cont+clicks_exp)
GC_sd_pooled=mt.sqrt(GC_pooled*(1-GC_pooled)*(1/clicks_cont+1/clicks_exp))
GC_ME=round(get_z_score(1-alpha/2)*GC_sd_pooled,4)
GC_diff=round(GC_exp-GC_cont,4)
print("The change due to the experiment is",GC_diff*100,"%")
print("Confidence Interval: [",GC_diff-GC_ME,",",GC_diff+GC_ME,"]")
print ("The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if",-GC["d_min"],"is not in the CI as well.")

The change due to the experiment is -2.06 %
Confidence Interval: [ -0.0235 , -0.0177 ]
The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if -0.01 is not in the CI as well.


In [50]:
#Net Conversion - number of payments divided by number of clicks
payments_cont=control["Payments"].sum()
payments_exp=experiment["Payments"].sum()

NC_cont=payments_cont/clicks_cont
NC_exp=payments_exp/clicks_exp
NC_pooled=(payments_cont+payments_exp)/(clicks_cont+clicks_exp)
NC_sd_pooled=mt.sqrt(NC_pooled*(1-NC_pooled)*(1/clicks_cont+1/clicks_exp))
NC_ME=round(get_z_score(1-alpha/2)*NC_sd_pooled,4)
NC_diff=round(NC_exp-NC_cont,4)
print("The change due to the experiment is",NC_diff*100,"%")
print("Confidence Interval: [",NC_diff-NC_ME,",",NC_diff+NC_ME,"]")
print ("The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if",NC["d_min"],"is not in the CI as well.")

The change due to the experiment is -0.49 %
Confidence Interval: [ -0.0072 , -0.0026 ]
The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if 0.0075 is not in the CI as well.


# **作業**
# 經由範例程式碼，熟悉A/B Test的步驟

請同學逐步跟隨程式了解A/B Test步驟

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
import seaborn as sns
import math as mt

In [162]:
# 建立函數
def z_value(alpha):
    return norm.ppf(alpha)

control=pd.read_csv("control_data.csv")
experiment=pd.read_csv("experiment_data.csv")
#Column包含Date、Pageviews、Clicks、Enrollments(有缺)、Payments(有缺)

#先蒐集測量值加總
pageviews_cont = control['Pageviews'].sum()
pageviews_exp = experiment['Pageviews'].sum()
clicks_cont = control['Clicks'].sum()
clicks_exp = experiment['Clicks'].sum()
enrollments_cont = control['Enrollments'].sum()
enrollments_exp = experiment['Enrollments'].sum()
payments_cont = control['Payments'].sum()
payments_exp = experiment['Payments'].sum()

print("pageviews_cont:",f'{pageviews_cont}',
      "\npageviews_exp:",f'{pageviews_exp}',
      "\nclicks_cont:",f'{clicks_cont}',
      "\nclicks_exp:",f'{clicks_exp}',
      "\nenrollments_cont:",f'{enrollments_cont}',
      "\nenrollments_exp:",f'{enrollments_exp}',
      "\npayments_cont:",f'{payments_cont}',
      "\npayments_exp:",f'{payments_exp}',)

pageviews_cont: 345543 
pageviews_exp: 344660 
clicks_cont: 28378 
clicks_exp: 28325 
enrollments_cont: 3785.0 
enrollments_exp: 3423.0 
payments_cont: 2033.0 
payments_exp: 1945.0


一、實驗組樣本數稍微多對照組一些但相當接近，但可試著驗證

In [180]:
# 驗證：
p = 0.5
alpha = 0.05
pageviews_total = pageviews_cont + pageviews_exp
p_ture = round(pageviews_cont/(pageviews_total),4)
sd = ( p*(1-p)/(pageviews_total) )**0.5
#加減標準常態sd*sd(從常態變成試算視角)
ME = round(z_value(1-(alpha/2))*sd,4)
#信賴區間
print ("pageviews:",p_ture,"有在信賴區間",p-ME,"與",p+ME,"的範圍內？")

p = 0.5
alpha = 0.05
clicks_total = clicks_cont + clicks_exp
p_ture2 = round(clicks_cont/(clicks_total),4)
sd2 = ( p*(1-p)/(clicks_total) )**0.5
ME2 = round(z_value(1-(alpha/2))*sd2,4)
print ("clicks:",p_ture2,"有在信賴區間",p-ME2,"與",p+ME2,"的範圍內？")

pageviews: 0.5006 有在信賴區間 0.4988 與 0.5012 的範圍內？
clicks: 0.5005 有在信賴區間 0.4959 與 0.5041 的範圍內？


都有符合，樣本數可認為沒有差異(對半分)

In [145]:
# 推算變數GC、NC、R
# 推算值：                 /      NConversion      \
# 邏輯架構：進入    →    點擊    →    註冊    →    付費
# 推算值：     \  CTP   / \GConversion/ \ Retention /

# pool機率：進行pool : p_pool = (X1+X2) / (N1+N2)

CTP_cont = clicks_cont/pageviews_cont
CTP_exp = clicks_exp/pageviews_exp
CTP_pooled = (clicks_cont + clicks_exp)/(pageviews_cont + pageviews_exp)

In [146]:
# 再驗證CTP、GC、NC、R作為推算變數，是否有新特徵(或是說結果有所不同)
alpha = 0.05
CTP_diff = round((CTP_exp - CTP_cont),4)
sd_pooled = ( CTP_pooled*(1-CTP_pooled)*(1/pageviews_cont+1/pageviews_exp) )**0.5
ME = round(z_value(1-(alpha/2))*sd_pooled,4)
print ("CTP_diff:",CTP_diff,"有在信賴區間",0-ME,"與",0+ME,"的範圍內？")

CTP_diff: 0.0001 有在信賴區間 -0.0013 與 0.0013 的範圍內？


In [147]:
#去除缺失值，Enrollments與payments不影響，僅需去除clicks的部分
clicks_cont=control["Clicks"].loc[control["Enrollments"].notnull()].sum()
clicks_exp=experiment["Clicks"].loc[experiment["Enrollments"].notnull()].sum()

GC_cont = enrollments_cont/clicks_cont
GC_exp = enrollments_exp/clicks_exp
GC_pooled = (enrollments_cont + enrollments_exp)/(clicks_cont + clicks_exp)

NC_cont = payments_cont/clicks_cont
NC_exp = payments_exp/clicks_exp
NC_pooled = (payments_cont + payments_exp)/(clicks_cont + clicks_exp)

R_cont = payments_cont/enrollments_cont
R_exp = payments_exp/enrollments_exp
R_pooled = (payments_cont + payments_exp)/(enrollments_cont + enrollments_exp)

#計算信賴區間
GC_diff = round((GC_exp - GC_cont),4)
sd_pooled2 = ( GC_pooled*(1-GC_pooled)*(1/clicks_cont+1/clicks_exp) )**0.5
ME2 = round(z_value(1-(alpha/2))*sd_pooled2,4)
print ("GC_diff:",GC_diff,"而",GC_diff-ME2,"與",GC_diff+ME2,"有包含零？")

NC_diff = round((NC_exp - NC_cont),4)
sd_pooled3 = ( NC_pooled*(1-NC_pooled)*(1/clicks_cont+1/clicks_exp) )**0.5
ME3 = round(z_value(1-(alpha/2))*sd_pooled3,4)
print ("NC_diff:",NC_diff,"而",NC_diff-ME3,"與",NC_diff+ME3,"有包含零？")

R_diff = round((R_exp - R_cont),4)
sd_pooled = ( R_pooled*(1-R_pooled)*(1/enrollments_cont+1/enrollments_exp) )**0.5
ME4 = round(z_value(1-(alpha/2))*sd_pooled,4)
print ("R_diff:",R_diff,"而",R_diff-ME4,"與",R_diff+ME4,"有包含零？")

GC_diff: -0.0206 而 -0.0292 與 -0.012 有包含零？
NC_diff: -0.0049 而 -0.0116 與 0.0018000000000000004 有包含零？
R_diff: 0.0311 而 0.0081 與 0.054099999999999995 有包含零？


在GC，對照組顯著比實驗組好
在NC，無顯著差異
在R，實驗組顯著比對照組好

當然也可以
數據集合併(cont.join(exp))、去除缺失值、建立比較的配對變數，以data['new'] = np.where(m1>m2,1,0)生成新列
計數不同列 x(為符合m1 > m2條件)，n為總數，另訂函數以計算 C n取x * 機率(如先前訂的是0.5)**x * (1-機率)**(n-x)
def prob(x, n):
    p = mt.factorial(n)/(mt.factorial(x)*mt.factorial(n-x))*0.5**x*0.5**(n-x)
    return p
以及
def two_tail_pvalue(x,n):
    p=0
    for i in range(0,x+1):
        p += get_prob(i,n)
    return 2*p
最後以two_tail_pvalue(x,n)來含括possibility(probability density)
會得出與pool相同的結果

# **作業 嘗試以函數算出樣本數**

## 解答一
可從四個變數中的三個（效應值、樣本數量、Type Ⅰ error水準(預設為0.05)、檢定力(1-Type Ⅱ error)），推導出第四個。
套用statsmodels.stats.api(power).NormalIndPower().solve_power(effect_size, power, alpha, ratio)\
以及statsmodels.stats.proportion.proportion_effectsize(比例1, 比例2)

In [182]:
effect_size = sms.proportion_effectsize(GC_cont-1*GC_diff, GC_cont+0*GC_diff)
required_n = sms.NormalIndPower().solve_power(
    effect_size, 
    power=0.8, 
    alpha=0.05, 
    ratio=1
    ) 
required_n = mt.ceil(required_n) #無條件進位

(effect_size,required_n)

(0.04902341935447052, 6532)

## 解答二
Tammy Rotem : 樣本數n是根據以下公式 

pow((z值1*標準差1 + z值2*標準差2),2) / pow(d,2)

In [185]:
# 根據H0 : Pcont = Pexp ; H1 : Pcont - Pexp = d，取得sd_cont, sd_exp
def count_sds(p,d):
    sd_0 = ( 2*p*(1-p) )**0.5
    sd_1 = ( p*(1-p) + (p+d)*(1-(p+d)) )**0.5
    sds = [sd_0, sd_1]
    return sds
def Sample_Number(sds,alpha,power,d): # alpha對應H0, 1-beta對應H1
    n = (z_value(1-alpha/2)*sds[0] + z_value(power)*sds[1])**2 / d**2
    return n

mt.ceil(Sample_Number(count_sds(GC_cont, GC_diff),0.05,0.8,GC_diff))

6258

# **作業** 自行開發雙樣本比例的信賴區間函數


## 解答一 : pdiff

In [216]:
def two_proprotions_confint(success_a, size_a, success_b, size_b, alpha):
    prop_a = success_a / size_a
    prop_b = success_b / size_b
    prop_pool = (success_a + success_b) / (size_a + size_b)
    prop_diff = prop_b - prop_a
    
    # standard formula for the confidence interval
    # point-estimtate +- z * standard_error
    se = np.sqrt( prop_a * (1 - prop_a) / size_a + prop_b * (1 - prop_b) / size_b ) 
    #等同於prop_pool*(1-prop_pool)*(1/size_a + 1/size_b)

    confidence = 1 - alpha
    #norm(loc=mean,scale=standard deviation)，等同前述的(1-alpha/2)
    z = norm.ppf(confidence + alpha / 2)
    CI = prop_diff + np.array([-1, 1]) * z * se
    return prop_diff, CI

two_proprotions_confint(enrollments_cont, clicks_cont, enrollments_exp, clicks_exp, 0.05)

(-0.012530660817334408, array([-0.01801297, -0.00704835]))

## 解答二 : 先pool

In [217]:
GC_cont = enrollments_cont/clicks_cont
GC_exp = enrollments_exp/clicks_exp
GC_pooled = (enrollments_cont + enrollments_exp)/(clicks_cont + clicks_exp)
prop_diff = GC_exp - GC_cont


def pdiff_CI(prop_pool, num_a, num_b, alhpa):
    se = (prop_pool*(1-prop_pool)*(1/num_a + 1/num_b))**0.5
    z = norm.ppf(1-alpha/2)
    return prop_diff, np.array([prop_diff-z*se, prop_diff+z*se])

pdiff_CI(GC_pooled, clicks_cont, clicks_exp, 0.05)

(-0.012530660817334408, array([-0.01801415, -0.00704717]))